In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import max

In [3]:
spark = SparkSession.builder.master("local").appName("load").getOrCreate()

In [4]:
def applyLatest(tq):
    latest_arrival = tq.groupBy("trade_dt", "symbol", "exchange", "event_tm", "event_seq_nb").agg(max("arrival_tm").alias("latest"))
    return tq.join(latest_arrival, ["trade_dt", "symbol", "exchange", "event_tm", "event_seq_nb"])

In [5]:
def clean(date, p):
    common_df = spark.read.parquet("c:/sb/equity-market-data-analysis/output_dir/{}/partition={}".format(date, p))
    #common_df.show()

    if p == 'T':
        trade = common_df.select("trade_dt", "symbol", "exchange", "event_tm", "event_seq_nb",
                                "arrival_tm", "trade_pr")
        corrected = applyLatest(trade)
        corrected.write.mode("append").parquet("c:/sb/equity-market-data-analysis/output_dir/trade/date={}".format(date))
    else:
        quote = common_df.select("trade_dt", "symbol", "exchange", "event_tm", "event_seq_nb",
                                "arrival_tm", "bid_pr", "bid_size", "ask_pr", "ask_size")
        corrected = applyLatest(quote)
        corrected.write.mode("append").parquet("c:/sb/equity-market-data-analysis/output_dir/quote/date={}".format(date))
#    print(common_df.count())
#    print(corrected.count())

In [6]:
#
clean('2020-08-05','Q')
clean('2020-08-05','T')
clean('2020-08-06','Q')
clean('2020-08-06','T')